# Prepare Sentinel-1 data obtained via GEE

## Libraries

In [1]:
#install.packages('gdalUtils')
library(gdalUtils)
library(raster)


Loading required package: sp


## Inputs

In [2]:
ifolder <- 'C:\\Users\\keers001\\OneDrive\ -\ WageningenUR\\RETURN\\Data\\RETURN\\20191019_SimulationOptSAR\\S1\\SampledData\\'
ofolder <- 'C:\\Users\\keers001\\OneDrive\ -\ WageningenUR\\RETURN\\Data\\RETURN\\20191019_SimulationOptSAR\\S1\\SampledData\\'
sfolder <- 'C:\\Users\\keers001\\OneDrive\ -\ WageningenUR\\RETURN\\Data\\RETURN\\20191019_SimulationOptSAR\\'

## Create regular time series

In [3]:
# Make a sequence of dates with daily interval
pdates <-seq(as.Date( '2015-01-01'),as.Date( '2019-12-31'),by='1 days',)
# directories of sampled rasters
sfolders <- list.files(path = ifolder, full.names = T, recursive = TRUE, include.dirs = T, pattern = 'drive*')
# sampled locations 
crds <- read.csv(file.path(sfolder, 'Split_RndmSample_NoFire_5_Tree_80_scl_30_npnt_20000.csv'),
        header=F)
slat <- crds$V4
slon <- crds$V3
coords = cbind(slon, slat)
sp = SpatialPoints(coords)
proj4string(sp) = CRS("+init=epsg:4326")


for (fo in 10:length(sfolders)){#:length(sfolders)
    ifiles <- list.files(paste0(sfolders[fo]))
    for (fi in 1:length(ifiles)){#:length(ifiles)
        # bounding box geotiff
        rs <- stack(file.path(sfolders[fo],ifiles[fi]))
        SPtrans = spTransform(sp, crs(rs))
        
        # observation dates
        met <- gdalinfo(file.path(sfolders[fo],ifiles[fi]))
        ind <- grep( ' Description = ', met)
        mp <- strsplit(gsub(' Description = ', '', met[ind]), '_')
        dts <- sapply(strsplit(sapply(mp, `[`, 6),'T'), `[`, 1)
        dts <- as.Date(dts,format = '%Y%m%d')
        #print(dts)
        
        # create regular time series for each sampled location
        tsi <- extract(rs, SPtrans, method='simple', buffer=NULL, small=FALSE, cellnumbers=FALSE, 
                       fun=NULL, na.rm=TRUE,1, df=FALSE, factors=FALSE)
        #print(dim(tsi))
        tsii <- cbind(slon,slat, tsi)
        pxi <- which(! rowSums(is.na(tsii[,-c(1,2)])) == (dim(tsii)[2]-2))
        tsii <- tsii[pxi,] # remove empty rows
        
        dfVI <- data.frame(data= matrix(NA, nrow = 1,ncol= (2+length(pdates))))#+length(pdates)
        colnames(dfVI) <- c('lon', 'lat', as.character(pdates))
        dfVI$lat <- as.numeric(tsii[2])
        dfVI$lon <- as.numeric(tsii[1])
          
        for (d in 1:length(pdates)){#
            dt<-mean(tsii[(2+which(dts == pdates[d]))])#
            dfVI[[as.character(pdates[d])]] <- dt    
          }
        head(dfVI)

        # export to RData file
        save(dfVI, file = file.path(ofolder,paste0(substr(ifiles[fi], 1, 6), 'pix_', pxi[1],'.rda')))
        rm(met, ind, mp, dts, rs, SPtrans, dfVI) 
        
        #tsii <- as.data.frame(tsii)
        #rownames(tsii) <- c('lon', 'lat', as.character(dts))
                
        #df <- data.frame(data= matrix(NA, nrow = 1, ncol= (2+length(pdates))))#+length(pdates)
        #colnames(df) <- c('lon', 'lat', as.character(pdates))
        #df[,c(1,2,(which(pdates %in% dts))+2)] <- as.matrix(tsii)

               
    }
}
